In [9]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Reading Session keys

In [10]:
session_key_list = pd.read_csv('session_keys.csv')

In [11]:
for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/session_result?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_result = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_result = pd.concat([df_result,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')


Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.
Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 ext

In [12]:
df_result

,position,driver_number,number_of_laps,points,dnf,dns,dsq,duration,gap_to_leader,meeting_key,session_key
0,1.0,1,57.0,25.0,False,False,False,5636.736,0,1141,7953
1,2.0,11,57.0,18.0,False,False,False,5648.723,11.987,1141,7953
2,3.0,14,57.0,15.0,False,False,False,5675.373,38.637,1141,7953
3,4.0,55,57.0,12.0,False,False,False,5684.788,48.052,1141,7953
4,5.0,44,57.0,10.0,False,False,False,5687.713,50.977,1141,7953
...,...,...,...,...,...,...,...,...,...,...,...
15,16.0,23,58.0,0.0,False,False,False,5257.796,90.327,1276,9839
16,17.0,6,57.0,0.0,False,False,False,NaN,+1 LAP,1276,9839
17,18.0,30,57.0,0.0,False,False,False,NaN,+1 LAP,1276,9839
18,19.0,10,57.0,0.0,False,False,False,NaN,+1 LAP,1276,9839


In [13]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [14]:
df_result.columns

Index(['position', 'driver_number', 'number_of_laps', 'points', 'dnf', 'dns',
       'dsq', 'duration', 'gap_to_leader', 'meeting_key', 'session_key'],
      dtype='object')

In [15]:
dtype_dict = {
    'position': types.Integer,          # finishing position; integer
    'driver_number': types.Integer,     # car/driver ID; integer
    'number_of_laps': types.Integer,    # completed laps; integer
    'points': types.Integer,              # fractional points sometimes occur
    'dnf': types.Boolean,               # did not finish
    'dns': types.Boolean,               # did not start
    'dsq': types.Boolean,               # disqualified
    'duration': types.Float,            # total time in seconds or similar numeric
    'gap_to_leader': types.String,       # time delta; float
    'meeting_key': types.Integer,        # identifiers stay string-ish
    'session_key': types.Integer         # same here
}

In [16]:
df_result.to_sql(name = 'raw_result', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

377